In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import os
import sys
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.exception_helpers import CapturedException

# Jupyter interactivity:
import os
import sys
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows

from pyphocorehelpers.exception_helpers import CapturedException
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult, BatchSessionCompletionHandler

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsHelpers

import IPython
from IPython.display import display, Javascript
from pyphocorehelpers.programming_helpers import IPythonHelpers, CodeParsers
from pyphocorehelpers.notebook_helpers import NotebookProcessor

Automatic pdb calling has been turned OFF


In [2]:
from notebook import notebookapp
import requests
from IPython.display import display, Javascript
import json

NotebookProcessor.get_running_notebook_path()
		
# print(notebookapp.list_running_servers())
# get_running_notebook_path()

jupyter_runtime_dir: "C:\Users\pho\AppData\Roaming\jupyter\runtime"
fullpath_connection_file: "c:\Users\pho\AppData\Roaming\jupyter\runtime\kernel-v2-26300DqCJ9A4V4vgo.json"
connection_file: "kernel-v2-26300DqCJ9A4V4vgo.json"
kernel_id: "v2-26300DqCJ9A4V4vgo"
jupyter_running_servers: "[]"


In [3]:
## Finds the path of THIS notebook:
notebook_path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve()
assert notebook_path.exists()
notebook_path

WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/SCRATCH/2023-11-13 - Programmatic ipynb Processing.ipynb')

In [6]:
# notebook_path = Path(r'../DOCUMENTATION_NOTEBOOK.ipynb').resolve()
notebook_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\ReviewOfWork_2024-07-30.ipynb').resolve()
assert notebook_path.exists()
processor = NotebookProcessor(path=notebook_path)


[{'cell_type': 'code', 'execution_count': None, 'id': 'initial_id', 'metadata': {'ExecuteTime': {'end_time': '2023-11-16T23:21:20.608442900Z', 'start_time': '2023-11-16T23:21:20.217442100Z'}, 'collapsed': True, 'notebookRunGroups': {'groupValue': '1'}}, 'outputs': [], 'source': '%config IPCompleter.use_jedi = False\n# %xmode Verbose\n# %xmode context\n%pdb off\n%load_ext autoreload\n%autoreload 3\n# !pip install viztracer\n%load_ext viztracer\nfrom viztracer import VizTracer\nimport sys\nfrom pathlib import Path\n\n# required to enable non-blocking interaction:\n%gui qt5\n\nimport importlib\nfrom copy import deepcopy\nfrom numba import jit\nimport numpy as np\nimport pandas as pd\npd.options.mode.chained_assignment = None  # default=\'warn\'\n# pd.options.mode.dtype_backend = \'pyarrow\' # use new pyarrow backend instead of numpy\nfrom attrs import define, field, fields, Factory, make_class\nimport tables as tb\nfrom datetime import datetime, timedelta\n\n# Pho\'s Formatting Preference

In [ ]:
processor.get_cells_with_tags()

In [8]:
run_groups = processor.get_cells_with_run_groups()
list(run_groups.keys()) # ['1', '2', '', '21']

run_groups['1']

[{'i': 0,
  'content': '%config IPCompleter.use_jedi = False\n# %xmode Verbose\n# %xmode context\n%pdb off\n%load_ext autoreload\n%autoreload 3\n# !pip install viztracer\n%load_ext viztracer\nfrom viztracer import VizTracer\nimport sys\nfrom pathlib import Path\n\n# required to enable non-blocking interaction:\n%gui qt5\n\nimport importlib\nfrom copy import deepcopy\nfrom numba import jit\nimport numpy as np\nimport pandas as pd\npd.options.mode.chained_assignment = None  # default=\'warn\'\n# pd.options.mode.dtype_backend = \'pyarrow\' # use new pyarrow backend instead of numpy\nfrom attrs import define, field, fields, Factory, make_class\nimport tables as tb\nfrom datetime import datetime, timedelta\n\n# Pho\'s Formatting Preferences\nimport builtins\n\nimport IPython\nfrom IPython.core.formatters import PlainTextFormatter\nfrom IPython import get_ipython\n\nfrom pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose\

In [9]:
cell_index_list = [0, 2]
for cell_index in cell_index_list:
	display(Javascript(f"Jupyter.notebook.execute_cells([{cell_index}])"))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
notebook_outline = processor.get_notebook_outline()
notebook_outline

In [ ]:

notebook_path = Path(r"C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/ReviewOfWork_2024-01-22.ipynb").resolve()
processor = NotebookProcessor(path=notebook_path)


In [ ]:

cells_with_tags = processor.get_cells_with_tags()
# print(cells_with_tags)

# Find cells with a specific tag
# specific_tag_cells = processor.get_cells_with_tag('specific_tag')
# print(specific_tag_cells)

empty_cells = processor.get_empty_cells()
# print(empty_cells)
len(empty_cells)



In [ ]:
## Remove all empty cells, and save the resultant notebook as the current notebook with the '_cleaned' filename suffix (but same extention)
new_path = processor.path.with_stem(f'{processor.path.stem}_cleaned').resolve()
processor.remove_empty_cells_and_save(new_path=new_path)


## Begin Usage: 

In [ ]:
# Example usage
notebook_path = Path(r"C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\ReviewOfWork_2024-01-22.ipynb").resolve()
# notebook_path = '../BatchGenerateOutputs_2023-11-13.ipynb'
extracted_cells = IPythonHelpers.extract_cells(notebook_path)
print(extracted_cells)


In [ ]:
cells_with_tags = []
for cell in extracted_cells:
	cell_content = cell['source']
	cell_tags = cell['metadata'].get('tags', None)
	if (cell_tags is not None) and (len(cell_tags) > 0):
		cells_with_tags.append({'content': cell_content, 'tags': cell_tags})

cells_with_tags

In [ ]:
from ipywidgets import get_ipython # required for IPythonHelpers.cell_vars
import io
from contextlib import redirect_stdout

ipy = get_ipython()
ipy

In [ ]:

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations, AcrossSessionTables

BATCH_DATE_TO_USE = '2023-11-15_GL' # used for filenames throught the notebook

active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default


In [ ]:


# ## Load the saved across-session results:
# Outputs: across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list, neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table

inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table

# `IPython` helpers Testing

In [ ]:
from IPython import get_ipython

# Get the IPython instance
ipython = get_ipython()

# Access a specific cell's content by its number
cell_number = 2  # Replace with the cell number you are interested in
cell_content = ipython.user_ns['In'][cell_number]

print(cell_content)


# `CodeParsers` testing

In [ ]:
from pyphocorehelpers.programming_helpers import CodeParsers
import ast

code_block: str = """
inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table
"""
print(CodeParsers.extract_assigned_variable_names(code_block)) # BUG: returning only: ['num_sessions', 'num_sessions']

print(CodeParsers.find_input_variables(code_block))
# print(extract_variable_names(code_block))

In [ ]:
code_block: str = '''
from pyphocorehelpers.print_helpers import generate_html_string
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import DisplayColorsEnum

formatted_title_strings_dict = DisplayColorsEnum.get_pyqtgraph_formatted_title_dict()
# formatted_title_strings_dict


""" 

Captures: active_epochs_df, rank_order_results, ripple_result_tuple, rank_order_results_debug_values
spikes_df
"""

decoder_name_to_column_name_prefix_map = dict(zip(['long_LR', 'long_RL', 'short_LR', 'short_RL'], ['LR_Long', 'RL_Long', 'LR_Short', 'RL_Short']))
ripple_combined_epoch_stats_df = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].ripple_combined_epoch_stats_df)

# def get_epoch_label_row(an_idx: int):
# 	a_label = active_epochs_df.label.to_numpy()[an_idx]
# 	a_row = active_epochs_df[active_epochs_df.label == a_label]
# 	return list(a_row.itertuples())[0], a_label

# def debug_plot_epoch_label_row(a_plotter, an_idx: int):
# 	a_row, a_label = get_epoch_label_row(an_idx=an_idx)
# 	print(f'debug_plot_epoch_label_row(an_idx: {an_idx}):\n\t{print_formatted_active_epochs_df(a_label, a_row)}')


def debug_print_dict_row(a_dict) -> str:
return '\t'.join([': '.join([str(k), f'{float(v):0.3f}']) for k, v in a_dict.items()])

def print_formatted_active_epochs_df(curr_epoch_label, corresponding_epoch_values_tuple):
""" 2023-12-13 - NEW - called to print the dataframe at a given index
	['start', 'stop', 'label', 'duration']
	['LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman']
	['LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman']
	
	['LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']
	['LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson']
	
	['LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells']
	['LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells']
"""
print(f'curr_epoch_label: {curr_epoch_label}') # ,\ncorresponding_epoch_values_tuple: {corresponding_epoch_values_tuple}\n'
# Extracting the fields
out_str = '\n'.join((
	debug_print_dict_row({field: getattr(corresponding_epoch_values_tuple, field) for field in ['Index', 'label', 'start', 'stop', 'duration']}),
	# debug_print_dict_row({field: getattr(corresponding_epoch_values_tuple, field) for field in ['LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman']}),
	debug_print_dict_row({field: getattr(corresponding_epoch_values_tuple, field) for field in ['LR_Long_spearman', 'LR_Short_spearman', 'RL_Long_spearman', 'RL_Short_spearman']}),
	# debug_print_dict_row({field: getattr(corresponding_epoch_values_tuple, field) for field in ['LR_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Long_Old_Spearman', 'RL_Short_Old_Spearman']}),
	# debug_print_dict_row({field: getattr(corresponding_epoch_values_tuple, field) for field in ['LR_Long_Z', 'LR_Short_Z', 'RL_Long_Z', 'RL_Short_Z']}),
	# debug_print_dict_row({field: getattr(corresponding_epoch_values_tuple, field) for field in ['LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman']}),
	# debug_print_dict_row({field: getattr(corresponding_epoch_values_tuple, field) for field in ['LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']}),
	# f'LR_relative_num_cells: {LR_relative_num_cells[an_idx]},\t\t RL_relative_num_cells: {RL_relative_num_cells[an_idx]}',
	# f'LR_template_epoch_actually_included_aclus: {corresponding_epoch_values_tuple.a[an_idx]},\nRL_template_epoch_actually_included_aclus: {RL_template_epoch_actually_included_aclus[an_idx]}',
	f'LR_Long rel: num_cells: {corresponding_epoch_values_tuple.LR_Long_rel_num_cells}, \t aclus: {corresponding_epoch_values_tuple.LR_Long_ActuallyIncludedAclus}', 
	f'RL_Long rel: num_cells: {corresponding_epoch_values_tuple.RL_Long_rel_num_cells}, \t aclus: {corresponding_epoch_values_tuple.RL_Long_ActuallyIncludedAclus}', 
))
print(out_str)



def debug_update_plot_titles(a_plotter, an_idx: int):
""" Updates the titles of each of the four rasters with the appropriate spearman rho value.
captures: rank_order_results_debug_values || active_epochs_df, formatted_title_strings_dict
"""
use_plaintext_title: bool = True

curr_epoch_label = a_plotter.lookup_label_from_index(an_idx)
curr_new_results_df = ripple_combined_epoch_stats_df[ripple_combined_epoch_stats_df.index == curr_epoch_label]

for a_decoder_name, a_root_plot in a_plotter.plots.root_plots.items():
	# a_real_value = rank_order_results_debug_values[a_decoder_name][0][an_idx]
	a_std_column_name: str = decoder_name_to_column_name_prefix_map[a_decoder_name]
	a_formatted_title_string_prefix: str = formatted_title_strings_dict[a_std_column_name]
	active_column_names = curr_new_results_df.filter(regex=f'^{a_std_column_name}').columns.tolist()
	# print(active_column_names)

	active_column_values = curr_new_results_df[active_column_names]
	active_values_dict = active_column_values.iloc[0].to_dict() # {'LR_Long_spearman': -0.34965034965034975, 'LR_Long_pearson': -0.5736588716389961, 'LR_Long_spearman_Z': -0.865774983083525, 'LR_Long_pearson_Z': -1.4243571733839517}
	active_raw_col_val_dict = {k.replace(f'{a_std_column_name}_', ''):v for k,v in active_values_dict.items()} # remove the "LR_Long" prefix so it's just the variable names
	
	active_formatted_col_val_list = [':'.join([generate_html_string(str(k), color='grey', bold=False), generate_html_string(f'{v:0.3f}', color='white', bold=True)]) for k,v in active_raw_col_val_dict.items()]
	final_values_string: str = '; '.join(active_formatted_col_val_list)

	if use_plaintext_title:
		title_str = generate_html_string(f"{a_std_column_name}: {final_values_string}")
	else:
		# Color formatted title:
		title_str = generate_html_string(f"{a_formatted_title_string_prefix}: {final_values_string}")
	
	a_root_plot.setTitle(title=title_str)


def debug_update_long_short_info_titles(a_plotter, an_idx: int):
""" Updates the titles of each of the four rasters with the appropriate spearman rho value.
captures: ripple_result_tuple, 
"""
has_long_short_info_labels = (hasattr(a_plotter.ui, 'long_info_label') and hasattr(a_plotter.ui, 'short_info_label'))
if has_long_short_info_labels:
	directional_likelihoods_tuple: DirectionalRankOrderLikelihoods = ripple_result_tuple.directional_likelihoods_tuple
	directional_likelihoods_tuple.long_best_direction_indices
	directional_likelihoods_tuple.short_best_direction_indices
	directional_likelihoods_tuple.long_relative_direction_likelihoods

	long_info_string_list = []

	if directional_likelihoods_tuple.long_relative_direction_likelihoods is not None:
		_long_LR_likelihood = directional_likelihoods_tuple.long_relative_direction_likelihoods[an_idx, 0]
		_long_RL_likelihood = directional_likelihoods_tuple.long_relative_direction_likelihoods[an_idx, 1]
		_long_LR_likelihood_str = generate_html_string(f'{float(_long_LR_likelihood):0.3f}', color='black', bold=True)
		_long_RL_likelihood_str = generate_html_string(f'{float(_long_RL_likelihood):0.3f}', color='black', bold=True)
		long_info_string_list.append(generate_html_string(f'LONG LR likelihood: {_long_LR_likelihood_str}'))
		long_info_string_list.append(generate_html_string(f'LONG RL likelihood: {_long_RL_likelihood_str}'))

	if directional_likelihoods_tuple.long_best_direction_indices is not None:
		_long_best_dir_IDX = directional_likelihoods_tuple.long_best_direction_indices[an_idx]		
		_long_best_dir_IDX_str = generate_html_string(f'{int(_long_best_dir_IDX)}', color='black', bold=True)
		long_info_string_list.append(generate_html_string(f'LONG best dir IDX: {_long_best_dir_IDX_str}'))
		
	# original:
	ripple_result_tuple.rank_order_z_score_df
	ripple_result_tuple.active_epochs
	
	long_best_dir_z_score_value_str = generate_html_string(f'{float(ripple_result_tuple.long_best_dir_z_score_values[an_idx]):0.3f}', color='black', bold=True)
	long_info_string_list.append(generate_html_string(f'LONG best_dir_z_score_value: {long_best_dir_z_score_value_str}'))

	short_best_dir_z_score_value_str = generate_html_string(f'{float(ripple_result_tuple.short_best_dir_z_score_values[an_idx]):0.3f}', color='black', bold=True)
	# j_str = generate_html_string('j', color='red', bold=True)
	a_plotter.ui.long_info_label.setText('\n'.join(long_info_string_list))
	a_plotter.ui.short_info_label.setText(generate_html_string(f'SHORT best_dir_z_score_value: {short_best_dir_z_score_value_str}'))
else:
	print(f'WARN: debug_update_long_short_info_titles(...) but plotter does not have the `a_plotter.ui.long_info_label`')
	

def plot_attached_directional_templates_pf_debugger():
curr_active_pipeline.reload_default_display_functions()
# epoch_active_aclus = np.array([9,  26,  31,  39,  40,  43,  47,  52,  53,  54,  60,  61,  65,  68,  72,  75,  77,  78,  81,  82,  84,  85,  90,  92,  93,  98, 102]) # some test indicies
epoch_active_aclus = None
_out_directional_template_pfs_debugger = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_template_debugger, included_any_context_neuron_ids=epoch_active_aclus, figure_name=f'<Controlled by RankOrderRastersDebugger>')

def debug_update_paired_directional_template_pfs_debugger(a_plotter, an_idx: int):
""" captures: _out_directional_template_pfs_debugger, """
epoch_active_aclus = deepcopy(a_plotter.get_epoch_active_aclus())
# update the displayed cells:
directional_template_pfs_debugger_on_update_callback = _out_directional_template_pfs_debugger.get('ui').on_update_callback
directional_template_pfs_debugger_on_update_callback(epoch_active_aclus)


import io
from contextlib import redirect_stdout # used by DocumentationFilePrinter to capture print output

def a_debug_callback_fn(a_plotter, an_idx: int, an_epoch=None):
global epoch_active_aclus, _out_directional_template_pfs_debugger
out = io.StringIO()
# _out.on_update_epoch_IDX(an_idx)

curr_epoch_label = a_plotter.lookup_label_from_index(an_idx)

# DO LR only:
LR_ranked_aclus_stats = rank_order_results.LR_ripple.ranked_aclus_stats_dict[curr_epoch_label]
LR_extra_info = rank_order_results.LR_ripple.extra_info_dict[curr_epoch_label]

RL_ranked_aclus_stats = rank_order_results.LR_ripple.ranked_aclus_stats_dict[curr_epoch_label]
RL_extra_info = rank_order_results.RL_ripple.extra_info_dict[curr_epoch_label]


# ripple_combined_epoch_stats_df = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].ripple_combined_epoch_stats_df)
curr_new_results_df = ripple_combined_epoch_stats_df[ripple_combined_epoch_stats_df.index == curr_epoch_label]
curr_new_results_df

# RL_extra_info_dict = rank_order_results.RL_ripple.extra_info_dict[curr_epoch_label]

with redirect_stdout(out):
	print(f'=====================================================================================\n\tactive_epoch_IDX: {an_idx} :::', end='\t')
	# print(f'')
	# print(f'LR_long_relative_real_values: {LR_long_relative_real_values[an_idx]:.4f},\t\t LR_long_relative_real_p_values: {LR_long_relative_real_p_values[an_idx]:.4f}')
	# print(f'LR_short_relative_real_values: {LR_short_relative_real_values[an_idx]:.4f},\t\t LR_short_relative_real_p_values: {LR_short_relative_real_p_values[an_idx]:.4f}')
	# print(f'LR_ripple.long_z_score: {rank_order_results.LR_ripple.long_z_score[an_idx]:.4f},\t\t LR_ripple.short_z_score: {rank_order_results.LR_ripple.short_z_score[an_idx]:.4f}')
	# print(f'RL_long_relative_real_values: {RL_long_relative_real_values[an_idx]:.4f},\t\t RL_long_relative_real_p_values: {RL_long_relative_real_p_values[an_idx]:.4f}')
	# print(f'RL_short_relative_real_values: {RL_short_relative_real_values[an_idx]:.4f},\t\t RL_short_relative_real_p_values: {RL_short_relative_real_p_values[an_idx]:.4f}')
	# print(f'RL_ripple.long_z_score: {rank_order_results.RL_ripple.long_z_score[an_idx]:.4f},\t\t RL_ripple.short_z_score: {rank_order_results.RL_ripple.short_z_score[an_idx]:.4f}')
	# print(f'LR_relative_num_cells: {LR_relative_num_cells[an_idx]},\t\t RL_relative_num_cells: {RL_relative_num_cells[an_idx]}')
	# print(f'LR_template_epoch_actually_included_aclus: {LR_template_epoch_actually_included_aclus[an_idx]},\nRL_template_epoch_actually_included_aclus: {RL_template_epoch_actually_included_aclus[an_idx]}')
	

	## Simple key-based indexing:
	print(f'')
	
	lr_long_columns = curr_new_results_df.filter(regex='^LR_Long').columns.tolist()
	# print(lr_long_columns)
	lr_long_columns_stripped = [col.replace('LR_Long_', '') for col in lr_long_columns]
	# print(lr_long_columns_stripped)

	print(f'curr_new_results_df:\n{curr_new_results_df[lr_long_columns]}')
	# print(render_dataframe(curr_new_results_df))
	# print(f'LR_long_relative_real_values: {LR_long_relative_real_values[an_idx]:.4f},\t\t LR_long_relative_real_p_values: {LR_long_relative_real_p_values[an_idx]:.4f}')
	# print(f'LR_short_relative_real_values: {LR_short_relative_real_values[an_idx]:.4f},\t\t LR_short_relative_real_p_values: {LR_short_relative_real_p_values[an_idx]:.4f}')
	# print(f'LR_ripple.long_z_score: {rank_order_results.LR_ripple.long_z_score[an_idx]:.4f},\t\t LR_ripple.short_z_score: {rank_order_results.LR_ripple.short_z_score[an_idx]:.4f}')
	# print(f'RL_long_relative_real_values: {RL_long_relative_real_values[an_idx]:.4f},\t\t RL_long_relative_real_p_values: {RL_long_relative_real_p_values[an_idx]:.4f}')
	# print(f'RL_short_relative_real_values: {RL_short_relative_real_values[an_idx]:.4f},\t\t RL_short_relative_real_p_values: {RL_short_relative_real_p_values[an_idx]:.4f}')
	# print(f'RL_ripple.long_z_score: {rank_order_results.RL_ripple.long_z_score[an_idx]:.4f},\t\t RL_ripple.short_z_score: {rank_order_results.RL_ripple.short_z_score[an_idx]:.4f}')

	# print(f'LR_relative_num_cells: {LR_extra_info[1]},\t\t RL_relative_num_cells: {RL_extra_info[1]}')
	# print(f'LR_template_epoch_actually_included_aclus: {len(LR_extra_info[1])},\nRL_template_epoch_actually_included_aclus: {len(RL_extra_info[1])}')


	# 'LR_Long_spearman', 'LR_Long_spearman_Z'
	## Extract Z-score variables:
	

	# """ captures: active_epochs_df, """
	# # corresponding_epoch_value = active_epochs_df.to_records()[active_epochs_df['label'] == curr_epoch_label]
	# corresponding_epoch_values_tuple = list(a_plotter.active_epochs_df[a_plotter.active_epochs_df['label'] == curr_epoch_label].itertuples(name='EpochRow'))[0] # EpochRow(Index=398, start=1714.3077712343074, stop=1714.6516814583447, label=409, duration=0.3439102240372449, end=1714.6516814583447, LR_Long_spearman=0.5269555552418339, RL_Long_spearman=-0.050011483546781, LR_Short_spearman=0.4606822127204283, RL_Short_spearman=-0.2035100246885261, LR_Long_pearson=0.4836286811698692, RL_Long_pearson=-0.003226348316225221, LR_Short_pearson=0.47186014640172635, RL_Short_pearson=-0.13444915290053647)
	# print_formatted_active_epochs_df(curr_epoch_label, corresponding_epoch_values_tuple)
	# LR_Long_pearson[active_epochs_df['label'] == curr_epoch_label].values
	# print(f'corresponding_epoch_value: {corresponding_epoch_value}')
	print(f'done.\n')
	# ripple_result_tuple.rank_order_z_score_df.label.to_numpy()[an_idx]
	# except BaseException as e:
	# 	print(f'ERR\n\ta_debug_callback_fn(...): e: {e}')
	# 	raise e
	# 	# pass
	

	# a_row, a_label = get_epoch_label_row(an_idx=an_idx)
	# print(f'!!!! debug_plot_epoch_label_row(an_idx: {an_idx}):\n\ta_label: {a_label}\n\ta_row: {a_row}')

	# display(LR_template_epoch_actually_included_aclus[an_idx])
	# display(RL_template_epoch_actually_included_aclus[an_idx])
	# epoch_active_aclus = np.sort(union_of_arrays(LR_template_epoch_actually_included_aclus[an_idx], RL_template_epoch_actually_included_aclus[an_idx]))
	# print(f'epoch_active_aclus: {epoch_active_aclus}')
	print(f'______________________________________________________________________________________________________________________\n')
	
a_plotter.write_to_log(str(out.getvalue()))
# # update the displayed cells:
# directional_template_pfs_debugger_on_update_callback = _out_directional_template_pfs_debugger.get('ui').on_update_callback
# directional_template_pfs_debugger_on_update_callback(epoch_active_aclus)
# _out_directional_template_pfs_debugger = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_template_debugger, included_any_context_neuron_ids=epoch_active_aclus)

# rank_order_results.ripple_most_likely_result_tuple.long_short_best_dir_z_score_diff_values[an_idx]
# display(LR_template_epoch_actually_included_aclus[an_idx])
# display(RL_template_epoch_actually_included_aclus[an_idx])




_out_rank_order_event_raster_debugger.on_idx_changed_callback_function_dict['a_debug_callback'] = a_debug_callback_fn
_out_rank_order_event_raster_debugger.on_idx_changed_callback_function_dict['debug_update_plot_titles_callback'] = debug_update_plot_titles
# _out_rank_order_event_raster_debugger.on_idx_changed_callback_function_dict['debug_update_paired_directional_template_pfs_debugger'] = debug_update_paired_directional_template_pfs_debugger
# _out_rank_order_event_raster_debugger.on_idx_changed_callback_function_dict['debug_update_long_short_info_titles'] = debug_update_long_short_info_titles
# _out_rank_order_event_raster_debugger.on_idx_changed_callback_function_dict['debug_plot_epoch_label_row'] = debug_plot_epoch_label_row



_out_rank_order_event_raster_debugger.on_update_epoch_IDX(11)
'''

In [ ]:
print(CodeParsers.extract_assigned_variable_names(code_block)) # BUG: returning only: ['num_sessions', 'num_sessions']

# print(CodeParsers.find_input_variables(code_block))
# print(extract_variable_names(code_block))

# Extract Embedded Images from Notebook Cells

In [ ]:
from pyphocorehelpers.programming_helpers import NotebookProcessor


notebook_path = Path(r'../DOCUMENTATION_NOTEBOOK.ipynb').resolve()
assert notebook_path.exists()
processor = NotebookProcessor(path=notebook_path)
processor

In [ ]:
cells_with_images = processor.get_cells_with_images()
cells_with_images

In [ ]:
extracted_cells = IPythonHelpers.extract_cells(notebook_path)
print(extracted_cells)

In [ ]:
from ipywidgets import get_ipython # required for IPythonHelpers.cell_vars
import io
from contextlib import redirect_stdout

ipy = get_ipython()
ipy

In [ ]:
import base64

# Example base64 string from notebook
base64_string = "iVBORw0KGgoAAAANSUhEUgAAA..."

# Decode the base64 string
image_data = base64.b64decode(base64_string)

# Save the decoded image to a file
with open("extracted_image.png", "wb") as image_file:
	image_file.write(image_data)


# 2024-08-06 - Programmatically Inspect my imported libraries to discover functions and methods decorated with `function_attributes`/`metadata_attributes`

In [ ]:
from pyphocorehelpers.programming_helpers import discover_classes_and_functions_with_custom_metadata, PythonPathStr, build_metadata_tags_database

# library_name = 'pyphocorehelpers'
library_name = 'pyphoplacecellanalysis'
discovered_with_metadata, result = discover_classes_and_functions_with_custom_metadata(library_name)
print('Classes:', result['classes'])
print('Functions:', result['functions'])
discovered_with_metadata

print(f'discovered_with_metadata.keys(): {list(discovered_with_metadata.keys())}')
df = pd.DataFrame.from_dict(discovered_with_metadata)
df
tag_database = build_metadata_tags_database(discovered_with_metadata=discovered_with_metadata)
tag_database

Classes: ['pyphoplacecellanalysis.General.Model.Configs.DynamicConfigs.InteractivePlaceCellConfig', 'pathlib.Path', 'pyphoplacecellanalysis.General.Model.Configs.DynamicConfigs.PlottingConfig', 'pyphoplacecellanalysis.General.Model.Configs.DynamicConfigs.VideoOutputModeConfig', 'pathlib.Path', 'pathlib.Path', 'neuropy.utils.mixins.print_helpers.ProgressMessagePrinter', 'matplotlib.colors.ListedColormap', 'pyphoplacecellanalysis.PhoPositionalData.plotting.visualization_window.VisualizationWindow', 'matplotlib.colors.BoundaryNorm', 'matplotlib.collections.BrokenBarHCollection', 'pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig.DisplayColorsEnum', 'pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.LapsVisualizationMixin.LapsVisualizationMixin', 'matplotlib.collections.LineCollection', 'matplotlib.colors.ListedColormap', 'pyphocorehelpers.gui.PyVista.PhoCustomVtkWidgets.PhoWidgetHelper', 'itertools.islice', 'pyphocorehelpers.gui.PyVista.CascadingDynamicPlotsLi

,pyphoplacecellanalysis.PhoPositionalData.analysis.neuronal_dimensionality_reduction.runAnalysis_PCAandICA,pyphoplacecellanalysis.PhoPositionalData.plotting.laps.plot_lap_trajectories_2d,pyphoplacecellanalysis.PhoPositionalData.plotting.laps.plot_lap_trajectories_3d,pyphoplacecellanalysis.PhoPositionalData.plotting.laps.plot_laps_2d,pyphoplacecellanalysis.PhoPositionalData.plotting.placefield.plot_1d_placecell_validations,pyphoplacecellanalysis.PhoPositionalData.plotting.placefield.plot_single_cell_1D_placecell_validation,pyphoplacecellanalysis.SpecificResults.AcrossSessionResults.AcrossSessionTables.build_custom_table,pyphoplacecellanalysis.SpecificResults.AcrossSessionResults.AcrossSessionTables.build_long_short_fr_indicies_analysis_table,pyphoplacecellanalysis.SpecificResults.AcrossSessionResults.AcrossSessionTables.build_neuron_identities_table,pyphoplacecellanalysis.SpecificResults.AcrossSessionResults.AcrossSessionTables.build_neuron_replay_stats_table,...,pyphoplacecellanalysis.SpecificResults.fourthYearPresentation.fig_remapping_cells,pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters.plot_multiple_raster_plot,pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots.FigureCollector,pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig.PlottingHelpers.helper_matplotlib_add_long_short_epoch_indicator_regions,pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig.PlottingHelpers.helper_plotly_add_long_short_epoch_indicator_regions,pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig.PlottingHelpers.helper_pyqtgraph_add_long_short_session_indicator_regions,pyphoplacecellanalysis.SpecificResults.fourthYearPresentation.export_active_relative_entropy_results_videos,pyphoplacecellanalysis.SpecificResults.fourthYearPresentation.fig_example_nontopo_remap,pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions.plot_long_short,pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions.plot_long_short_any_values
kind,fcn,fcn,fcn,fcn,fcn,fcn,fcn,fcn,fcn,fcn,...,fcn,fcn,fcn,fcn,fcn,fcn,fcn,fcn,fcn,fcn
short_name,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,plot_long_short,plot_long_short_any_values
tags,"[dimensionality-reduction, unused, ICA, PCA]","[lap, trajectories, 2D, matplotlib, plotting, ...","[lap, trajectories, 3D, pyvista, qt, multiplot...","[lap, trajectories, 2D, matplotlib, plotting, ...","[plotting_fn, display_fn, pf1D]","[pf1D, 1D]",[],[],[],[],...,[display],"[pyqtgraph, raster, 2D]","[collector, figure, output]","[matplotlib, epoch, region]","[plotly, helper, long_short, regions, rectangles]","[pyqtgraph, helper, long_short, regions, recta...","[surprise, video, export]",[display],"[pyqtgraph, long_short]","[pyqtgraph, long_short]"
creation_date,2024-03-28 02:49,2023-05-09 05:13,2023-05-09 05:13,2023-05-09 05:13,2023-12-07 13:58,2023-09-06 01:55,2023-08-25 14:28,2023-08-25 14:28,2023-08-25 14:28,2023-08-25 14:28,...,2024-06-10 21:39,2023-06-16 20:45,2024-06-05 17:27,2023-12-23 17:52,2024-01-29 23:03,2023-04-19 19:04,2023-09-27 20:54,2024-06-10 21:40,2023-04-19 17:26,2023-03-28 17:27
input_requires,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[pyphoplacecellanalysis.Pho2D.PyQtPlots.Extens...,[],[],[],[]
output_provides,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
uses,[],[],[],"[plot_position_curves_figure, _plot_helper_add...",[plot_single_cell_1D_placecell_validation],"[plot_placefield_tuning_curve, active_epoch_pl...",[],[],[],[],...,[_display_long_short_pf1D_comparison],"[_prepare_spikes_df_from_filter_epochs, _plot_...",[],[],[],[],[],[_display_long_short_pf1D_comparison],[],[]
used_by,[],[],[],[estimation_session_laps],[DefaultDisplayFunctions._display_1d_placefiel...,[plot_1d_placecell_validations],[

In [29]:
tag_database['transition_matrix']

{'pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions.TrackTemplates.compute_decoder_transition_matricies'}